## Initialize the project 

In [1]:

from typing import Dict, List, Optional, Union, Callable, Literal
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.formatting_utils import colored
from typing_extensions import Annotated
import autogen
from autogen import Agent
from autogen.token_count_utils import count_token, get_max_token_limit
from autogen.agentchat.contrib.capabilities import transform_messages, transforms

from teachability import Teachability
from concurrent.futures import ThreadPoolExecutor, as_completed

import arxiv

import requests


import os
import shutil
import requests
import pickle
import re
from pathlib import Path

import nest_asyncio
nest_asyncio.apply()

import sqlite3
from utils import *

In [2]:
print(autogen.__version__)

0.2.29


### parameters

In [3]:
version = "0.0.1"
ProjectID = "Quantum_Com"
initiate_db = False

# config_file = "OAI_CONFIG_LIST"
config_file = "OAI_CONFIG_LIST-sweden-505"
max_round = 10
silent = False
recall_threshold = 0.7

Project_dir = Path(f"./{ProjectID}/{version}")
output_dir = f'{Project_dir}/pdf_output'
db_dir = f'{Project_dir}/memo-db/'

config_list = autogen.config_list_from_json(
    config_file,
    file_location=".",
    filter_dict={
        "model": ["gpt-4-32k", "gpt-4"]#, "gpt4", "gpt-35-turbo-16k", "gpt-4-0613", "gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613"]
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

config_list_32 = autogen.config_list_from_json(
    config_file,
    file_location=".",
    filter_dict={
        "model": ["gpt-4-32k"]
    },
)

config_list_4v = autogen.config_list_from_json(
    config_file,
    file_location=".",
    filter_dict={
        "model": ["gpt-4v"]
    },
)

config_list_35 = autogen.config_list_from_json(
    config_file,
    file_location=".",
    filter_dict={
        "model": ["gpt-35-turbo-16k", "gpt-35-turbo"]
    },
)

print("LLM models 32K: ", [config_list_32[i]["model"] for i in range(len(config_list_32))])

# Configuration for the Language Model (LLM)
llm_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 120,
    # "seed": 42,
}

# Configuration for the Language Model (LLM)
llm_config_32 = {
    "config_list": config_list_32,  # config_list should be defined or imported
    "timeout": 120,
    # "seed": 42,
}

llm_config_35 = {
    "config_list": config_list_35,  # config_list should be defined or imported
    "timeout": 120,
    # "seed": 42,
}

# Configuration for the manager using the same config_list as llm_config
manager_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 60,
    # "seed": 42,
}

# Termination message definition
termination_msg = (
    lambda x: isinstance(x, dict)
    and "TERMINATE" in str(x.get("content", "")).upper()
)



LLM models:  ['gpt-4', 'gpt-4-32k']
LLM models 32K:  ['gpt-4-32k']


In [4]:
import requests
import os
import cairosvg

def call_kroki_and_save_convert(
        diagram_source: Annotated[str, "The source code for the diagram"],
        diagram_type: Annotated[str, "The type of diagram to generate. it support bpmn, pikchr, nwdiag, c4plantuml, rackdiag, dot, symbolator, d2, tikz, mermaid, erd, graphviz, vegalite, ditaa, umlet, diagramsnet, plantuml, seqdiag, nomnoml, wavedrom, structurizr, bytefield, wireviz, excalidraw, dbml, packetdiag, svgbob, vega or blockdiag"],
        file_name: Annotated[str, "The name of the file to save the diagram to"] = "result"
) -> str:
    # Define the URL of the Kroki API
    url = "https://kroki.io/"

    # Define the payload for the POST request
    payload = {
        "diagram_source": diagram_source,
        "diagram_type": diagram_type,
        "output_format": "svg"
    }
    
    # Define the headers for the POST request
    headers = {
        'Content-Type': 'application/json'
    }

    try:
        # Send the POST request
        response = requests.post(url, json=payload, headers=headers)

        # Check if the request was successful
        if response.status_code == 200:
            # Ensure the diagrams folder exists
            os.makedirs(f'{Project_dir}/diagrams', exist_ok=True)

            # Define the path for the SVG file
            svg_path = f'{Project_dir}/diagrams/{file_name}.svg'
            # Define the path for the PNG file
            png_path = f'{Project_dir}/diagrams/{file_name}.png'
            # define the path for markdown file
            md_path = f'{Project_dir}/diagrams/{file_name}.md'

            # check if the files exist and change the name
            i = 0
            while os.path.exists(svg_path):
                svg_path = f'{Project_dir}/diagrams/{file_name}_{i}.svg'
                png_path = f'{Project_dir}/diagrams/{file_name}_{i}.png'
                md_path = f'{Project_dir}/diagrams/{file_name}_{i}.md'

            # Save the SVG content to a file
            with open(svg_path, 'w') as file:
                file.write(response.text)

            # Convert SVG to PNG using cairosvg
            cairosvg.svg2png(url=svg_path, write_to=png_path)

            # Save the markdown content to a file
            with open(md_path, 'w') as file:
                file.write(f"![{file_name}]({png_path})")
                file.write("\n\n")
                file.write(f"```{diagram_type}\n{diagram_source}\n```")

            return f"Dear manager, the image Successfully created and saved diagram as PNG: {png_path}"
        else:
            return f"Failed to created SVG: HTTP Status {response.status_code} - {response.text}"

    except requests.exceptions.RequestException as e:
        return f"HTTP Request failed: {e}"

    except FileNotFoundError as e:
        return f"File not found: {e}"

    except OSError as e:
        return f"OS error: {e}"

    except Exception as e:
        return f"An error occurred: {e}"

# You can call the function like this:
args = {"diagram_source":"digraph PolymerRepresentationEvolution {\n  rankdir=LR;\n  node [shape=box];\n\n  Wu2020 [label=\"Wu et al., 2020\\nPolymer Informatics & ML\", shape=ellipse, style=filled, fillcolor=lightblue, URL=\"http://arxiv.org/pdf/2010.15166v1\"];\n  Antoniuk2022 [label=\"Antoniuk et al., 2022\\nPeriodic Graph & MPNN\", shape=ellipse, style=filled, fillcolor=lightgreen, URL=\"http://arxiv.org/pdf/2311.14744v1\"];\n\n  OpenDatabases [label=\"Need for open databases\", shape=note];\n  LearnedDescriptors [label=\"Shift to Learned Descriptors\", shape=note];\n  GlobalCollaboration [label=\"International Collaboration\", shape=note];\n\n  Wu2020 -> Antoniuk2022 [label=\"20% error reduction\", color=red, fontcolor=red];\n  Wu2020 -> OpenDatabases [style=dotted];\n  Antoniuk2022 -> LearnedDescriptors [style=dotted];\n\n  FutureAdvancements [label=\"Future Trends\", shape=none, fontcolor=grey, fontsize=10];\n\n  subgraph cluster_collab {\n    style=invis;\n    GlobalMap [label=\"\", shape=none, image=\"global_collab.png\"];\n  }\n\n  Antoniuk2022 -> FutureAdvancements [style=dashed, color=grey];\n\n  Antoniuk2022 -> GlobalCollaboration [style=dotted, arrowhead=none];\n\n  {rank=same; Wu2020 -> Antoniuk2022 [style=invis]}\n}","diagram_type":"graphviz","file_name":"polymer_representation_evolution"}
# call_kroki_and_save_convert(**args)


In [5]:
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent

def visital_feedback(
        image_path: Annotated[str, "The path to the image file"],
) -> str:
    
    file_name = os.path.basename(image_path)
    image_path = f'{Project_dir}/diagrams/{file_name}'
    
    user = autogen.UserProxyAgent(
        name="user",
        human_input_mode="NEVER",
        is_termination_msg=termination_msg,
        code_execution_config={
            "last_n_messages": 2,
            "work_dir": "./image_generation",
            "use_docker": False,
            "timeout": 60
        }
    )
    
    critics = MultimodalConversableAgent(
            name="Critics", 
            system_message="""Criticize the input figure. How to replot the figure so it will be better? Find bugs and issues for the figure.
            Pay attention to the color, format, and presentation. Keep in mind of the reader-friendliness.
            If you think the figures is good enough, then simply say NO_ISSUES""",
            llm_config={"config_list": config_list_4v, "max_tokens": 300},
            human_input_mode="NEVER",
            max_consecutive_auto_reply=1,
            #     use_docker=False,
        )
    
    user.send(
                message=f"give feedback to this image creator to improve <img {image_path}>",
                recipient=critics,
                request_reply=True,
            )

    feedback = user._oai_messages[critics][-1]["content"]
    if feedback.find("NO_ISSUES") >= 0:
        return "Terminat. No issues found."
    
    return f"Here is the feedback to your figure {image_path}: {feedback}"

# visital_feedback('./diagrams/research_process_flow.png')

In [6]:
prompt = """carefully read the following section of a paper with the title {title}:
{section}

to enhance the readibility of this section, generate at least a diagram related to the content of the section.
if you need more information, please get help from your MEMOS.
"""

title = "Exploring the Intricacies of Polymer Representation: Unraveling Complexity"

section = """
**Case Study: Discovery of Thermally Conductive Polymers**

A significant study in polymer informatics is the 'Machine Learning-Assisted Exploration of Thermally Conductive Polymers Based on High-Throughput Molecular Dynamics Simulations' by Ruimin Ma et al. (2021). This research underscores how ML, combined with molecular dynamics simulations, accelerates the design process to identify polymers with high thermal conductivity, which are pivotal for heat transfer applications. By leveraging the expansive PolyInfo database, the research team demonstrated a paradigm shift from labor-intensive experiments to a data-enriched, algorithmic approach (Ma et al., 2021).
"""

message = prompt.format(title=title, section=section)

In [7]:
from autogen.agentchat.contrib.agent_builder import AgentBuilder

def generate_image(title, section):
    builder = AgentBuilder(config_file_or_env=config_file)
    _, agent_configs = builder.load("./visualization_agents.json") 
    agent_configs = agent_configs["agent_configs"]

    teachability = Teachability(
                                verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
                                reset_db=False,
                                path_to_db_dir=db_dir,
                                recall_threshold=recall_threshold,  # Higher numbers allow more (but less relevant) memos to be recalled.
                                llm_config=llm_config_35,
                                )

    user = autogen.UserProxyAgent(
            name="user",
            human_input_mode="NEVER",
            is_termination_msg=termination_msg,
            code_execution_config={
                "last_n_messages": 2,
                "work_dir": "./image_generation",
                "use_docker": False,
                "timeout": 60
            }
        )

    agents = {}
    for agent in agent_configs:
        agent_name = agent["name"]
        agents[agent_name] = autogen.ConversableAgent(
                                        name=agent["name"],
                                        is_termination_msg=termination_msg,
                                        system_message=agent["system_message"],
                                        llm_config=llm_config,
                                        description=agent["description"],
                                    )
        
        teachability.add_to_agent(agents[agent_name])
        autogen.agentchat.register_function(
                call_kroki_and_save_convert,
                caller=agents[agent_name],
                executor=user,
                name="call_kroki_and_save_convert",
                description="Generate a diagram from a given source code and save it as a PNG file.",
            )
        
        autogen.agentchat.register_function(
                call_kroki_and_save_convert,
                caller=agents[agent_name],
                executor=user,
                name="call_kroki_and_save_convert",
                description="Generate a diagram from a given source code and save it as a PNG file.",
            )
        
    critics = autogen.AssistantAgent(
        name="Critics",
        system_message="Please provide feedback on the generated figure by calling the `visital_feedback` function.",
        llm_config=llm_config,
        human_input_mode="NEVER",
        is_termination_msg=termination_msg,
        max_consecutive_auto_reply=1,
    )

    autogen.agentchat.register_function(
            visital_feedback,
            caller=critics,
            executor=user,
            name="visital_feedback",
            description="Provide feedback on the generated figure.",
        )

    groupchat = autogen.GroupChat(
        agents=[user] + list(agents.values()),
        messages=[],
        speaker_selection_method="auto",  # With two agents, this is equivalent to a 1:1 conversation.
        allow_repeat_speaker=True,
        max_round=max_round,
    )

    manager = autogen.GroupChatManager(
                groupchat=groupchat,
                is_termination_msg=termination_msg,
                llm_config=manager_config,
                code_execution_config={
                    "work_dir": "coding",
                    "use_docker": False,
                },
            )
    
    section_title = section[:-4].split("/")[-1].split("_")[1].strip()
    with open(section, "r") as f:
        section_content = f.read()
    message = prompt.format(title=title, section=section_content)
    chat_hist = user.initiate_chat(manager, 
                                    silent=silent, 
                                    message=message,
                                    )
    
    images_list = []
    # if chat_hist[:]["content"] start with "Dear manager, the image Successfully created and saved diagram as PNG:
    for msg in chat_hist.chat_history:
        if msg["content"].startswith("Dear manager, the image Successfully created and saved diagram as PNG:"):
            images_list.append(msg["content"].split(": ")[1])

    images = []
    for img in images_list:

        # rename the image path to {section_title}-{image_name}.png
        img_name = f"{section_title}-{os.path.basename(img)}".replace(" ", "_")
        img_path = f"{Project_dir}/diagrams/{img_name}"
        os.rename(img, img_path)
        images.append(img_path)

        # add image to the section
        section_content += f"\n\n![{section_title}](./diagrams/{img_name})"
        section_md = section[:-4] + ".md"
        # save the updated section to the file
        with open(section_md, "w") as f:
            f.write(section_content)
        

    return images
    

In [8]:
import glob
title = "Exploring the Intricacies of Polymer Representation: Unraveling Complexity"

sections = glob.glob(f"{Project_dir}/section_*.txt")


with ThreadPoolExecutor() as executor:
        futures = [executor.submit(generate_image, title, section) for section in sections]
        for future in as_completed(futures):
            try:
                images = future.result()
                print(images)
            except Exception as e:
                print(f"Error: {e}")
                continue

Loading config from ./visualization_agents.json
==> Creating agents...
Creating agent content_analyst_visual_designer with backbone gpt-4...


/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading config from ./visualization_agents.json
==> Creating agents...
Creating agent content_analyst_visual_designer with backbone gpt-4...
Creating agent image_developer with backbone gpt-4...
Loading config from ./visualization_agents.json
==> Creating agents...
Creating agent content_analyst_visual_designer with backbone gpt-4...
Loading config from ./visualization_agents.json
==> Creating agents...
Creating agent content_analyst_visual_designer with backbone gpt-4...
Creating agent image_creator with backbone gpt-4...
Creating agent image_developer with backbone gpt-4...
Creating agent image_developer with backbone gpt-4...
Loading config from ./visualization_agents.json
==> Creating agents...
Creating agent content_analyst_visual_designer with backbone gpt-4...
Loading config from ./visualization_agents.json
==> Creating agents...
Creating agent content_analyst_visual_designer with backbone gpt-4...
Adding user console proxy...
Creating agent image_developer with backbone gpt-4..

/home/autogen/autogen/autogen/agentchat/conversable_agent.py:2489: UserWarning: Function 'call_kroki_and_save_convert' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)


    Location = Quantum_Com/0.0.1/memo-db/uid_text_dict.pkl


...


>>>>>>>> EXECUTING FUNCTION call_kroki_and_save_convert...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...

>>>>>>>> USING AUTO REPLY...
image_creator (to chat_manager):

The diagram related to the content of the section "Exploring the Intricacies of Polymer Representation: Unraveling Complexity" has been successfully created. Here is the representation of the evolution and ethical aspects of Quantum Material Science:

![Quantum Material Science Evolution](sandbox:/Quantum_Material_Science_Evolution.png)

--------------------------------------------------------------------------------
image_creator (to chat_manager):

The diagram has been successfully generated and should visually represent the challenges in material science pertaining to quantum computing as outlined in the revised text of the paper. Please find the diagram file here:

![Quantum Material Science Challenges](sandbox:/Quantum_Material_Science_Challenges.png)

If you have any additional requests or need furt

[runtime logging] log_function_use: autogen logger is None



>>>>>>>> EXECUTING FUNCTION call_kroki_and_save_convert...


[runtime logging] log_function_use: autogen logger is None


user (to chat_manager):

user (to chat_manager):

***** Response from calling tool (call_De7E6hmyMuIGrLDvwUSBJddi) *****
Failed to created SVG: HTTP Status 400 - Error 400: Error: <stdin>: syntax error in line 5 near '{'
 (exit code 1)
**********************************************************************

--------------------------------------------------------------------------------
user (to chat_manager):

***** Response from calling tool (call_8deHoDL2rarXazy5JVuUBjWw) *****
Dear manager, the image Successfully created and saved diagram as PNG: Quantum_Com/0.0.1/diagrams/Technical_Advancements_Quantum.png
**********************************************************************

--------------------------------------------------------------------------------

Next speaker: image_creator


>>>>>>>> USING AUTO REPLY...
image_creator (to chat_manager):

It appears that we have encountered an error while trying to create one of the diagrams. Specifically, the first diagram hit a syntax e